In [1]:
# VERSION
from sys import version
import torch
torch.manual_seed(42)
version, torch.__version__

('3.10.8 (main, Nov 30 2022, 20:34:09) [Clang 14.0.0 (clang-1400.0.29.202)]',
 '2.0.0')

In [2]:
# INDEXING WITH LISTS & TENSORS IN ONE DIMENSION
t = torch.randn((2,5))
t, t[1], t[[0,0,1]], t[torch.tensor([0,1,1])]

(tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
         [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674]]),
 tensor([-0.1863,  2.2082, -0.6380,  0.4617,  0.2674]),
 tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
         [ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
         [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674]]),
 tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
         [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674],
         [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674]]))

In [3]:
# INDEXING WITH TENSORS IN MULTIPLE DIMENSIONS
x = torch.arange(12).reshape(3, 4)
y = torch.tensor([[0, 2], [1, 1], [2, 0]])
x, y, y[:, 0], y[:, 1], x[y[:, 0], y[:, 1]]  # do you see it for the last one?

(tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[0, 2],
         [1, 1],
         [2, 0]]),
 tensor([0, 1, 2]),
 tensor([2, 1, 0]),
 tensor([2, 5, 8]))

In [4]:
# more fun example from karpathy https://youtu.be/TCH_1BHY58I?t=1028
C = torch.randn((27,2))
X = torch.arange(32*3).reshape((32,3)) % 27
C.shape, X.shape, C[X].shape
# if you understand this, you can explain why the % 27 is needed

(torch.Size([27, 2]), torch.Size([32, 3]), torch.Size([32, 3, 2]))

In [5]:
# CONCATENATE
X = torch.randn((2,3))
X, torch.cat((x, x), 0), torch.cat((x, x), 1)

(tensor([[ 0.5568, -0.8123,  1.1964],
         [ 0.8613, -1.3682, -0.7740]]),
 tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[ 0,  1,  2,  3,  0,  1,  2,  3],
         [ 4,  5,  6,  7,  4,  5,  6,  7],
         [ 8,  9, 10, 11,  8,  9, 10, 11]]))

In [6]:
# UNBIND
t = torch.arange(6).reshape((2,3))
t, t.unbind(), t.unbind(1)

(tensor([[0, 1, 2],
         [3, 4, 5]]),
 (tensor([0, 1, 2]), tensor([3, 4, 5])),
 (tensor([0, 3]), tensor([1, 4]), tensor([2, 5])))

In [7]:
t2 = torch.arange(12).reshape((2,3,2))
t2, t2.unbind()

(tensor([[[ 0,  1],
          [ 2,  3],
          [ 4,  5]],
 
         [[ 6,  7],
          [ 8,  9],
          [10, 11]]]),
 (tensor([[0, 1],
          [2, 3],
          [4, 5]]),
  tensor([[ 6,  7],
          [ 8,  9],
          [10, 11]])))

In [8]:
# MICROGRAD https://github.com/karpathy/micrograd/blob/master/test/test_engine.py
# from micrograd.engine import Value

# x = Value(-4.0)
# z = 2 * x + 2 + x
# q = z.relu() + z * x
# h = (z * z).relu()
# y = h + q + q * x
# y.backward()
# xmg, ymg = x, y

x = torch.Tensor([-4.0]).double()
x.requires_grad = True
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x
y.backward() # manual backprod https://youtu.be/VMj-3S1tku0?t=1929
# xpt, ypt = x, y

# forward pass went well
# assert ymg.data == ypt.data.item()
# backward pass went well
# assert xmg.grad == xpt.grad.item()
x, y, x.grad.item(), x.data.item()

(tensor([-4.], dtype=torch.float64, requires_grad=True),
 tensor([-20.], dtype=torch.float64, grad_fn=<AddBackward0>),
 46.0,
 -4.0)

In [9]:
# print(y.grad)
# /var/folders/8f/pmq_g1556d942470cc0ghx480000gn/T/ipykernel_73007/486760323.py:1: UserWarning:
# The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute
# won't be populated during autograd.backward(). If you indeed want the .grad field to be populated
# for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor
# by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531
# for more informations. (Triggered internally at 
# /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)


In [10]:
# NON-LEAFS
# Only leaf Tensors will have their grad populated during a call to backward().
x.is_leaf, y.is_leaf, h.is_leaf, q.is_leaf, z.is_leaf
# some off-hand micrograd talk about leaf nodes https://youtu.be/VMj-3S1tku0?t=1748

(True, False, False, False, False)

In [11]:
a = torch.rand(10, requires_grad=True)
b = torch.rand(10, requires_grad=True).double()
c = torch.rand(10, requires_grad=True) + 2
d = torch.rand(10).double()
e = torch.rand(10).double().requires_grad_()
f = torch.rand(10, requires_grad=True, dtype=torch.double)
a.is_leaf, b.is_leaf, c.is_leaf, d.is_leaf, e.is_leaf, f.is_leaf


(True, False, False, True, True, True)

In [12]:
b = torch.rand(10, requires_grad=True) # .double()
b.is_leaf # False

True

In [13]:
# VIEWS
a = torch.arange(18)
a, a.view(2, 9), a.view(-1, 9).shape, a.view(2,3,3)

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
         [ 9, 10, 11, 12, 13, 14, 15, 16, 17]]),
 torch.Size([2, 9]),
 tensor([[[ 0,  1,  2],
          [ 3,  4,  5],
          [ 6,  7,  8]],
 
         [[ 9, 10, 11],
          [12, 13, 14],
          [15, 16, 17]]]))

In [14]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # ya that's a global lol
a.storage() # karpathy did his lectures before torch 2.0!
# read more about pytorch internals here! :
# http://blog.ezyang.com/2019/05/pytorch-internals/

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [15]:
# MEMORY DATA POINTER
t.data_ptr(), t.view(-1, 3).data_ptr()

(5437794688, 5437794688)